<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Time Series Forecasting on Number of Passengers for FlyHigh Airline</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider a startup airline <b>FlyHigh Airlines</b> as our client. The client requires us to forecast the number of passengers who will fly using their airline. The ARIMA model has been widely utilized in many fields for forecasting since it is recognized to be reliable, efficient, and capable of predicting short-term share market movements. Hence, we'll build an ARIMA model to forecast the demand(passenger traffic) in Airplanes.
<br>
<br>
ARIMA(Auto Regressive Integrated Moving Average) is a combination of 2 models AR(Auto Regressive) and MA(Moving Average). It has 3 hyperparameters - P(auto regressive lags), d(order of differentiation), Q(moving avg.) which respectively comes from the AR, I and MA components. The AR part is correlation between prev and current time periods. To smooth out the noise, the MA part is used. The I part binds together the AR and MA parts.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Data:</b></p>
<p style = 'font-size:16px;font-family:Arial'>The data for this demonstration resides on Vantage. The data consists of monthly flight totals by month between 2007 and 2018  (144 rows) which we selected to simply be able to understand the functionality. The same functionality can be applied to huge datasets with multiple series concurrently.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Connect to Vantage</li>    
    <li>Explore the dataset</li>
    <li>Check for Stationarity using Dickey-Fuller Test</li>
    <li>Make series stationary using TD_DIFF (D)</li>
    <li>Check for autocorrelation of the time series (Q)</li>
    <li>Check for partial autocorrelation of the time series (P)</li>
    <li>Using ARIMA(AutoRegressive Integrated Moving Average) model to forecast number of passengers</li>
        <ul>
            <li>7.1 Estimation step using TD_ARIMAESTIMATE</li>
            <li>7.2 Extract residuals</li>
            <li>7.3 Create table PLOT_ESTIMATE for plotting</li>
            <li>7.4 Validation step using TD_ARIMAVALIDATE</li>
            <li>7.5 Extract residuals</li>
            <li>7.6 Create table PLOT_VALIDATE for plotting</li>
            <li>7.7 Forecast step using TD_ARIMAFORECAST</li>
            <li>7.8 Create table PLOT_FORECAST for plotting</li>
        </ul>
    <li>Cleanup</li>
</ol>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Accessing the Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>This demo works with data imported from CSV using another notebook and creates a table under Demo_User. You may need to use the Data Dictionary "Manage Your Space" routine to cleanup tables you no longer need. 
    
<p style = 'font-size:16px;font-family:Arial'>Use the link below to access the data preparation notebook.

[Click Here to get data for this notebook](../Data_Dictionary/Data_Dictionary.ipynb#TRNG_AirPassengers)

[Click Here to Manage Your Space](../Data_Dictionary/Data_Dictionary.ipynb#Manage_Your_Space)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>This step connects with Vantage. You might be prompted to enter the password.</p>

In [1]:
%connect local

Password: ···


Success: 'local' connection established and activated for user 'demo_user'


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Explore the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>The data is classified in date/time and the passengers travelling per month. Time series deals with 2 columns, one is temporal i.e. <b>Date</b> in this case and another is the value to be forecasted i.e. <b>Passengers</b>. Following is a sample of the dataset.</p>

In [2]:
SELECT TOP 5 * FROM TRNG_AirPassengers.airpassengers ORDER BY "Date";

,Date,Passengers,seriesID
1,2007-01-01 01:33:36.000000,112,FlyHigh
2,2007-02-01 01:33:36.000000,118,FlyHigh
3,2007-03-01 01:33:36.000000,132,FlyHigh
4,2007-04-01 01:33:36.000000,129,FlyHigh
5,2007-05-01 01:33:36.000000,121,FlyHigh


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the total passengers by month from 2007 to 2018.</p>

<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [3]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(TRNG_AirPassengers.airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS("Passengers"),CONTENT(REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('line'),
            LEGEND('upper left'),
            TITLE('Number of Passengers Travelling Monthly')
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph that looks like follows:</p>
<img id="fig1" src="images/fig1.png" alt="Number of Passengers Travelling Monthly" width="400" />
<p style = 'font-size:16px;font-family:Arial'>This shows that the data has yearly cycles and an upward trend.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Check for Stationarity using Dickey-Fuller Test</b>
<br>
<b style = font-size:16px;font-family:Arial>What is Stationarity?</b>
<br>
<p style = 'font-size:16px;font-family:Arial'>Before applying any statistical model on a Time Series, the series has to be stationary, which means that, over different time periods,
<br>
a) It should have constant mean.
<br>
b) It should have constant variance or standard deviation.
<br>
c) Auto-covariance should not depend on time.
</p>
<p style = 'font-size:16px;font-family:Arial'> We can visually examine if the mean and variance are constant over different periods of time or not. Alternatively, Dickey-Fuller test is widely used to check for stationarity in the time series data.
<br>

<p style = 'font-size:16px;font-family:Arial'>The following query would check for stationarity by using Dickey-Fuller Test. If the Null Hypothesis of the Dickey-Fuller Test:
<br>
• <b>ACCEPT</b> means the null hypothesis is accepted. The process
is <b>non-stationary</b> and unit roots are present.
<br>
• <b>REJECT</b> means the null hypothesis is rejected. The process is
<b>stationary</b>, and no unit roots are present.</p>

In [4]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
    SERIES_SPEC(
        TABLE_NAME(TRNG_AirPassengers.airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        ALGORITHM('TREND')
    )
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,FlyHigh,0,144,TREND ,-4.639196964346928,0.0008857695636920808,ACCEPT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column named NULL_HYP, we observe that the NULL Hypothesis is accepted and hence the series is Non-Stationary. This suggests that we need to make this series stationary.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Make series stationary using TD_DIFF</b>
<p style = 'font-size:16px;font-family:Arial'>TD_DIFF transforms a stationary, seasonal, or nonstationary time series into a differenced time series. We are going to create a new table which contains differenced series and then check if the differenced series is stationary or not. Here, the parameter <b>LAG</b> is set to 12(12 units of time i.e. 12 months or 1 year) in order to consider the cyclical nature of the data. We'll check if the newly created table(which contains differenced series) is stationary or not(Using Dickey-Fuller test). If not, we'll again use TD_DIFF recursively till the series becomes stationary.</p>

In [5]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Passengers)
INTO VOLATILE ART(diff1_air)
TD_DIFF(
    SERIES_SPEC(
            TABLE_NAME(TRNG_AirPassengers.airpassengers),
            ROW_AXIS(TIMECODE("Date")),
            SERIES_ID("seriesID"),
            PAYLOAD(
                FIELDS("Passengers"),
                CONTENT(REAL)
            )
    ),
    FUNC_PARAMS(
          LAG(12),
          DIFFERENCES(4),
          SEASONAL_MULTIPLIER(0)
    )
);

Success: 96 rows affected

In [6]:
SELECT TOP 5 * FROM diff1_air ORDER BY ROW_I;

,seriesID,ROW_I,Passengers
1,FlyHigh,0,34
2,FlyHigh,1,-10
3,FlyHigh,2,100
4,FlyHigh,3,78
5,FlyHigh,4,150


<p style = 'font-size:16px;font-family:Arial'>In the above result, <b>ROW_I</b> is Row identifier of the ordered result sets and <b>OUT_Passengers</b> is Transformed magnitudes of differenced time series elements.
    <br>
    <br>
The following cell applies Dickey-Fuller test to check for stationarity.</p>

In [7]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        ALGORITHM('TREND')
    )
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,FlyHigh,0,96,TREND ,-3.1342412241637474,0.09831876940084541,REJECT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column NULL_HYP, we observe that the NULL Hypothesis is rejected and hence the series is stationary. Differencing the series once gave us stationary series. Hence the value of order of differentiation or <b>D = 1</b>. This value will be used further in ARIMA_ESTIMATE.</p>
<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Check for autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>TD_ACF calculates the autocorrelation or autocovariance of a time series. The autocorrelation and autocovariance show how the time series covaries or correlates with itself when delayed by a lag in time or space. Here we check autocorrelation with a maximum lag of 12 time steps.</p>

In [8]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Auto_Correlation)
INTO VOLATILE ART(ACFDemo)
TD_ACF(
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        FUNC_TYPE(0),   -- Use 0 for autocorrelation, or 1 for autocovariance
        DEMEAN(1),
        QSTAT(0),
        ALPHA(0.05)
    )
);

Success: 13 rows affected

In [9]:
SELECT TOP 5 * FROM ACFDemo ORDER BY ROW_I;

,seriesID,ROW_I,Auto_Correlation,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,FlyHigh,0,1,0,1,1
2,FlyHigh,1,0.736849780698874,0.2000379865148046,0.5368117941840693,0.9368877672136786
3,FlyHigh,2,0.6092948162493547,0.2889074338100545,0.32038738243930015,0.8982022500594091
4,FlyHigh,3,0.41962593972958917,0.3364193839941808,0.08320655573540836,0.75604532372377
5,FlyHigh,4,0.3221378707614005,0.35674953103167856,-0.034611660270278066,0.678887401793079


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Auto Correlation Plot.</p>
<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [10]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(ACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(Auto_Correlation, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph that looks like follows:</p>
<img id="fig2" src="images/fig2.png" alt="Auto Correlation" width="400" />
<p style = 'font-size:16px;font-family:Arial'>To get the value of Moving Average or Q, we need the lag(here, ROW_I) where the value from the ACF plot is just outside the significant limit. Looking at the graph, the Auto-Correlation value at ROW_I = 3 is outside the confidence band and is very much closer to it. Hence it is acceptable to say that the value of Moving Average or <b>Q = 3</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Check for partial autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>The TD_PACF function provides insight as to whether the function being modeled is stationary or not. The partial auto correlations are used to measure the degree of correlation between time series sample points. Here we check partial autocorrelation with a maximum lag of 12 time steps.</p>

In [11]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Partial_Auto_Correlation)
INTO VOLATILE ART(PACFDemo)
TD_PACF (
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ) ,
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        ALGORITHM(LEVINSON_DURBIN),
        ALPHA(0.05)
    )
);

Success: 13 rows affected

In [12]:
SELECT TOP 5 * FROM PACFDemo ORDER BY ROW_I;

,seriesID,ROW_I,Partial_Auto_Correlation,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,FlyHigh,0,1,0.2000379865148046,1,1
2,FlyHigh,1,0.736849780698874,0.2000379865148046,0.5368117941840693,0.9368877672136786
3,FlyHigh,2,0.145163261004824,0.2000379865148046,-0.0548747255099806,0.34520124751962855
4,FlyHigh,3,-0.1589643249713953,0.2000379865148046,-0.3590023114861999,0.0410736615434093
5,FlyHigh,4,0.036264086522446906,0.2000379865148046,-0.1637738999923577,0.23630207303725148


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Partial Auto Correlation Plot.</p>
<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [13]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(PACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(Partial_Auto_Correlation, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph that looks like follows:</p>
<img id="fig3" src="images/fig3.png" alt="Partial Auto Correlation" width="400" />
<p style = 'font-size:16px;font-family:Arial'>To get the value of Auto Regressive lags or P, we need the lag(here, Row_I) where the value from the PACF plot falls just outside the significant limit. Looking at the graph, the Partial Auto-Correlation value at ROW_I = 1 falls outside the significant limit. Hence we can say that the value of Auto Regressive lags or <b>P = 1</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. Using ARIMA (AutoRegressive Integrated Moving Average) model to forecast number of passengers</b>
<p style = 'font-size:16px;font-family:Arial'>
ARIMA functions on VANTAGE run in the following order:
<br>
1. Run <b>TD_ARIMAESTIMATE</b> function to get the coefficients for the ARIMA model.
<br>
2. <i>[Optional]</i> Run <b>TD_ARIMAVALIDATE</b> function to validate the the "goodness of fit" of the ARIMA model, when
FIT_PERCENTAGE is not 100 in TD_ARIMAESTIMATE.
<br>
3. Run the <b>TD_ARIMAFORECAST</b> function with input from step 1 or step 2 to forecast the future periods
beyond the last observed period.
</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.1 Estimation step using TD_ARIMAESTIMATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAESTIMATE function estimates the coefficients corresponding to an ARIMA model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the TD_ARIMAVALIDATE and TD_ARIMAFORECAST functions. This function is for
univariate series.</p>

<br>

<p style = 'font-size:16px;font-family:Arial'>Here, the previously estimated parameters P, d and Q need to be passed in the MODEL_ORDER(P, d, Q) i.e <b>MODEL_ORDER(1, 1, 3)</b>. The output is stored in an ART(Analytical Result Table). Fit percentage is 80, meaning the ARIMA model is being trained on 80% of the data. Remaining 20% of data will be used to validate the model.</p>

In [14]:
EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
TD_ARIMAESTIMATE(
    SERIES_SPEC(
        TABLE_NAME(TRNG_AirPassengers.airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL))),
     FUNC_PARAMS(
        NONSEASONAL(MODEL_ORDER(1, 1, 3)),
        SEASONAL(MODEL_ORDER(1, 1, 3), PERIOD(12)),
        CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
        RESIDUALS(1), ALGORITHM(CSS_MLE),  FIT_PERCENTAGE(80)
    )
);

Success: 158 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.2 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART(Analytical Result Table). Here we extract the residuals from the ART of previous estimation step. Analytical Result Tables have multiple layers which store different data. The default layer has model information. The ARTFITRESIDUALS layer has information about actual, calculated and residual values. The ARTFITMETADATA layer has performance metrics.
</p>

In [15]:
CREATE TABLE AR_RESIDUALS AS (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(ART_EST),
            LAYER(ARTFITRESIDUALS)
        )
    )
) WITH DATA;

Success: 0 rows affected

In [16]:
SELECT TOP 5 * FROM AR_RESIDUALS ORDER BY ROW_I;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,FlyHigh,0,112,111.93533678259698,0.06466321740301692
2,FlyHigh,1,118,117.9664341620197,0.033565837980311194
3,FlyHigh,2,132,131.96619386853047,0.03380613146952975
4,FlyHigh,3,129,128.97744816594982,0.022551834050185737
5,FlyHigh,4,121,120.9892461371173,0.010753862882700261


<p style = 'font-size:16px;font-family:Arial'>The output above shows the actual value, calculated value and the residual of the ARIMA model. The actual values indicate the actual number of passengers flying as per the input dataset. The Calculated value indicates what the model has learned during the estimate phase. The residual value is the difference between actual and calculated value.
<br>
<br>
In the following cell, we extract additional metrics from the estimate phase i.e. TD_ARIMAESTIMATE.
</p>

In [17]:
SELECT * FROM (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(ART_EST),
            LAYER(ARTFITMETADATA)
        )
    )
) AS T;

,seriesID,ROW_I,NUM_SAMPLES,VAR_COUNT,R_SQUARE,R_ADJ_SQUARE,STD_ERROR,STD_ERROR_DF,ME,MAE,MSE,MPE,MAPE,FSTAT_CALC,P_VALUE,NUM_DF,DENOM_DF,SIGNIFICANCE_LEVEL,F_CRITICAL,F_CRITICAL_P,NULL_HYPOTH
1,FlyHigh,0,115,8,0.991091251376211,0.9905084360456827,75.642224556571,113,0.6063622781093305,6.422070579408627,74.32670760776107,0.2532102004518045,2.677695324371721,1700.5236469636197,1.379855387071475e-106,7,107,0.05,2.0963213718883407,0.05,REJECT


<p style = 'font-size:16px;font-family:Arial'>The output shown above shows performance metric which can be used to understand how well has the ARIMA Model trained. For example, the R_SQUARED value above is 0.99 which suggests that our model fits the data really well. In general, the higher the R_SQUARED value, the better the model fits to the data. R_SQUARED value ranges from 0 to 1.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.3 Create table PLOT_ESTIMATE for plotting</b></p>
</p>
<p style = 'font-size:16px;font-family:Arial'>Here, we'll create a table which will be used to plot the actual and estimated time series.</p>

In [18]:
CREATE TABLE PLOT_ESTIMATE (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [19]:
INSERT INTO PLOT_ESTIMATE SELECT 'FlyHigh', ROW_I, ACTUAL_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 
INSERT INTO PLOT_ESTIMATE SELECT 'ESTIMATED', ROW_I, CALC_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 

Success: 0 rows affected

Success: 226 rows affected

In [20]:
SELECT TOP 5 * FROM PLOT_ESTIMATE ORDER BY ROW_I;

,DatasetID,ROW_I,FIT_MAGNITUDE
1,ESTIMATED,2,131.96619386853047
2,FlyHigh,2,132
3,ESTIMATED,3,128.97744816594982
4,FlyHigh,3,129
5,FlyHigh,4,121


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Actual and Estimated values by the fitted ARIMA model.</p>
<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [21]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_ESTIMATE),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FlyHigh"},{"DatasetID":"ESTIMATED"}]'),
        PAYLOAD(
            FIELDS(FIT_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA ESTIMATE'),
        PLOTS[
            (
                TITLE ('ORIGINAL and ESTIMATED SERIES'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph looks like follows:</p>
<img id="fig4" src="images/fig4.png" alt="ARIMA Estimate" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line indicates the actual number of passengers travelled and blue line indicates the estimated number of passengers travelled. This graph shows how well the ARIMA model has learned on the training dataset.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.4 Validation step using TD_ARIMAVALIDATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAVALIDATE function provides data scientists with a metrics collection for model selection and the produced residuals, such that a number of model validation tests can be performed. The TD_ARIMAVALIDATE function performs in-sample forecasting for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), and ARIMA models. It also supports the extended BoxJenkins seasonal ARIMA model formula.
</p>
<p style = 'font-size:16px;font-family:Arial'>We'll use the output of previous estimation step to validate the model. The train-validate split here for the dataset is 80:20. Hence 20% of data will be used for validation of the estimated model.</p>

In [22]:
EXECUTE FUNCTION 
INTO VOLATILE ART(AR_VALIDATE)
TD_ARIMAVALIDATE(
    ART_SPEC(TABLE_NAME(ART_EST)),
    FUNC_PARAMS(
        FIT_METRICS(1),
        RESIDUALS(1)
    )
);

Success: 36 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.5 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table output of previous validation step.
</p>

In [23]:
CREATE TABLE AR_VALIDATE_RESIDUALS AS (
    EXECUTE FUNCTION 
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(AR_VALIDATE),
            LAYER(ARTFITRESIDUALS)
        )
    )
) WITH DATA;

Success: 0 rows affected

In [24]:
SELECT TOP 5 * FROM AR_VALIDATE_RESIDUALS ORDER BY ROW_I;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,FlyHigh,102,505,504.71541606026983,0.2845839397301829
2,FlyHigh,103,404,403.9536743499105,0.04632565008948548
3,FlyHigh,104,359,359.00708485101035,-0.007084851010358043
4,FlyHigh,105,310,310.0491891859232,-0.0491891859232049
5,FlyHigh,106,337,337.01793845433355,-0.01793845433357069


<p style = 'font-size:16px;font-family:Arial'>The output above shows the actual value, calculated value and the residual of the ARIMA model for the validation phase. The actual value is number of passengers from the unseen data i.e. validation data. The calculated value is the predicted value by the ARIMA model on the unseen data.
<br>
<br>
In the following step, we again pull the metadata from the ART which is the output of the validation phase.
</p>

In [25]:
SELECT * FROM (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(AR_VALIDATE),
            LAYER(ARTFITMETADATA)
        )
    )
) AS T;

,seriesID,ROW_I,NUM_SAMPLES,VAR_COUNT,R_SQUARE,R_ADJ_SQUARE,STD_ERROR,STD_ERROR_DF,ME,MAE,MSE,MPE,MAPE,F_STAT,F_STAT_P,NUM_DF,DENOM_DF,SIGNIFICANCE_LEVEL,F_CRITICAL,F_CRITICAL_P,NULL_HYPOTH
1,FlyHigh,0,29,8,0.9666565962324857,0.9555421283099809,232.09637845124456,27,-1.5998317732158143,8.90800243068664,216.08973166150355,-0.38991795942872737,1.9713669208207807,86.97281803973569,4.520858091149696e-14,7,21,0.05,2.4875777037220397,0.05,REJECT翾


<p style = 'font-size:16px;font-family:Arial'>The output above shows the performace metrics for the validation phase. These metrics help us in analyzing how well our model perfoms on unseen dataset i.e. validation dataset. For example, the R_SQUARED value from the above output is 0.97 which is a good value. In general, the higher the R_SQUARED value the better the model is. R_SQUARED value ranges from 0 to 1.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.6 Create table PLOT_VALIDATE for plotting</b></p>
</p>
<p style = 'font-size:16px;font-family:Arial'>Here, we'll create a table which will be used to plot the actual and validated time series.</p>

In [26]:
CREATE TABLE PLOT_VALIDATE (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [27]:
INSERT INTO PLOT_VALIDATE SELECT 'FlyHigh', ROW_I, ACTUAL_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0; 
INSERT INTO PLOT_VALIDATE SELECT 'PREDICTED', ROW_I, CALC_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0;

Success: 0 rows affected

Success: 58 rows affected

<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Actual and Predicted values by ARIMA model.</p>
<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [28]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_VALIDATE),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FlyHigh"},{"DatasetID":"PREDICTED"}]'),
        PAYLOAD(
            FIELDS(FIT_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA VALIDATE'),
        PLOTS[
            (
                TITLE ('ORIGINAL and PREDICTED SERIES'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph that looks like follows:</p>
<img id="fig5" src="images/fig5.png" alt="ARIMA Validate" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line indicates the actual number of passengers travelled and blue line indicates the predicted number of passengers travelled. This graph shows how well the ARIMA model predicts on validation data.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.7 Forecast step using TD_ARIMAFORECAST</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAFORECAST function is used to forecast a user-defined number of periods based on
models fitted from the TD_ARIMAESTIMATE function.</p>
<p style = 'font-size:16px;font-family:Arial'>Here in the following cell, we use the estimated and validated model to forecast number of passengers for next 6 periods i.e. next 6 months.</p>

In [29]:
EXECUTE FUNCTION INTO VOLATILE ART(ARMA_FORECAST)
TD_ARIMAFORECAST(
           ART_SPEC(TABLE_NAME(AR_VALIDATE)),
           FUNC_PARAMS(FORECAST_PERIODS(6)));

Success: 6 rows affected

In [30]:
SELECT * FROM ARMA_FORECAST;

,seriesID,ROW_I,FORECAST_VALUE,LO_80,HI_80,LO_95,HI_95
1,FlyHigh,0,448.80286816861485,437.05108030679287,460.55465603043683,430.8300655474117,466.775670789818
2,FlyHigh,1,422.45446369929124,407.3437819074814,437.5651454911011,399.3446778088999,445.56424958968256
3,FlyHigh,2,470.4832626708105,452.0389194713592,488.92760587026186,442.27508331127655,498.6914420303445
4,FlyHigh,3,506.76582797603453,486.9446633054949,526.5869926465742,476.4519825820672,537.0796733700018
5,FlyHigh,4,530.0389185693377,509.4219846866439,550.6558524520316,498.50804959136406,561.5697875473114
6,FlyHigh,5,570.6776110806936,549.4849395681031,591.8702825932842,538.2662276733071,603.0889944880802


<p style = 'font-size:16px;font-family:Arial'>The above output shows us the forecasted value for the next 6 months. Observe that we also have forecated values with 80% and 95% confidence as well.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.8 Create table PLOT_FORECAST for plotting</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here, we'll create a table which will be used to plot the forecasted number of passengers in the next 6 months.</p>

In [31]:
CREATE TABLE PLOT_FORECAST (DatasetID VARCHAR(16), ROW_I BIGINT, FORECAST_MAGNITUDE FLOAT);

Success: 0 rows affected

In [32]:
INSERT INTO PLOT_FORECAST   SELECT 'FORECASTED', ROW_I, FORECAST_VALUE FROM ARMA_FORECAST; 
INSERT INTO PLOT_FORECAST   SELECT 'UPPER_BOUND', ROW_I, HI_80 FROM ARMA_FORECAST ; 
INSERT INTO PLOT_FORECAST   SELECT 'LOWER_BOUND', ROW_I, LO_80 FROM ARMA_FORECAST ; 

Success: 0 rows affected

Success: 0 rows affected

Success: 18 rows affected

In [33]:
SELECT * FROM PLOT_FORECAST ORDER BY ROW_I;

,DatasetID,ROW_I,FORECAST_MAGNITUDE
1,UPPER_BOUND,0,460.55465603043683
2,LOWER_BOUND,0,437.05108030679287
3,FORECASTED,0,448.80286816861485
4,UPPER_BOUND,1,437.5651454911011
5,LOWER_BOUND,1,407.3437819074814
6,FORECASTED,1,422.45446369929124
7,UPPER_BOUND,2,488.92760587026186
8,LOWER_BOUND,2,452.0389194713592
9,FORECASTED,2,470.4832626708105
10,UPPER_BOUND,3,526.5869926465742


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Forecasted values by ARIMA model.</p>
<i>* Please <b> right click on the cell under the IMAGE column </b> from the output and choose view image to see the plot generated. </i>

In [34]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_FORECAST),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FORECASTED"},{"DatasetID":"UPPER_BOUND"},{"DatasetID":"LOWER_BOUND"}]'),
        PAYLOAD(
            FIELDS(FORECAST_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA FORECAST'),
        PLOTS[
            (
                TITLE ('Forecast'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       ),
                        (
                        ID(3),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph that looks like follows:</p>
<img id="fig6" src="images/fig6.png" alt="ARIMA Forecast" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line is Forecasted number of passengers for the next 6 months and blue lines are the upper and lower confidence interval with 80% confidence level.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have trained and validated the ARIMA model on the air passengers dataset and the results match closely with the actual data. The goodness of fit metrics calculated in estimate and validate phase also resonate with our understanding that the model is well trained in order to forecast.</p>

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>8. Cleanup</b>
<p style = 'font-size:16px;font-family:Arial'>This section drops all the tables created during the demonstration.</p>

In [35]:
DROP TABLE diff1_air;

Success: 0 rows affected

In [36]:
DROP TABLE ACFDemo;

Success: 0 rows affected

In [37]:
DROP TABLE PACFDemo;

Success: 0 rows affected

In [38]:
DROP TABLE ART_EST;

Success: 0 rows affected

In [39]:
DROP TABLE AR_RESIDUALS;

Success: 19 rows affected

In [40]:
DROP TABLE PLOT_ESTIMATE;

Success: 17 rows affected

In [41]:
DROP TABLE AR_VALIDATE;

Success: 0 rows affected

In [42]:
DROP TABLE AR_VALIDATE_RESIDUALS;

Success: 19 rows affected

In [43]:
DROP TABLE PLOT_VALIDATE;

Success: 17 rows affected

In [44]:
DROP TABLE ARMA_FORECAST;

Success: 0 rows affected

In [45]:
DROP TABLE PLOT_FORECAST;

Success: 17 rows affected

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>UAF(Unbounded Array Framework) Documentation: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework'>https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework</a></li>
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>